In [1]:
! pip install nltk
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [2]:
# Tokenization using NLTK
from nltk import word_tokenize, sent_tokenize
sent = "GeeksforGeeks is a great learning platform.\
It is one of the best for Computer Science students."
print(word_tokenize(sent))
print(sent_tokenize(sent))

['GeeksforGeeks', 'is', 'a', 'great', 'learning', 'platform.It', 'is', 'one', 'of', 'the', 'best', 'for', 'Computer', 'Science', 'students', '.']
['GeeksforGeeks is a great learning platform.It is one of the best for Computer Science students.']


In [3]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
print(porter.stem("play"))
print(porter.stem("playing"))
print(porter.stem("plays"))
print(porter.stem("played"))

play
play
play
play


In [4]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()
print(porter.stem("Communication"))

commun


In [5]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("plays", 'v'))
print(lemmatizer.lemmatize("played", 'v'))
print(lemmatizer.lemmatize("play", 'v'))
print(lemmatizer.lemmatize("playing", 'v'))

play
play
play
play


In [6]:
from nltk import pos_tag
from nltk import word_tokenize

text = "GeeksforGeeks is a Computer Science platform."
tokenized_text = word_tokenize(text)
tags = tokens_tag = pos_tag(tokenized_text)
tags

[('GeeksforGeeks', 'NNP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('Computer', 'NNP'),
 ('Science', 'NNP'),
 ('platform', 'NN'),
 ('.', '.')]

In [7]:
from nltk import word_tokenize, pos_tag, ne_chunk

# Download the required resource for NER
nltk.download('maxent_ne_chunker_tab')
nltk.download('words') # This resource is also needed for the chunker

# Sample text
text = "Barack Obama was born in Hawaii in 1961."

# Tokenize and POS tag the sentence
tokens = word_tokenize(text)
tags = pos_tag(tokens)

# Apply Named Entity Recognition
entities = ne_chunk(tags)
print(entities)

[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


(S
  (PERSON Barack/NNP)
  (PERSON Obama/NNP)
  was/VBD
  born/VBN
  in/IN
  (GPE Hawaii/NNP)
  in/IN
  1961/CD
  ./.)


In [14]:
!pip install vaderSentiment transformers scikit-learn --quiet

In [15]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load IMDb reviews (pre-tokenized)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

# Decode word index to text
word_index = imdb.get_word_index()
index_word = {v + 3: k for k, v in word_index.items()}
index_word[0], index_word[1], index_word[2] = "<PAD>", "<START>", "<UNK>"

def decode_review(encoded_review):
    return " ".join([index_word.get(i, "?") for i in encoded_review])

# Convert to readable DataFrame (first 2000 samples)
import pandas as pd

reviews = [decode_review(x) for x in X_train[:2000]]
labels = y_train[:2000]

df = pd.DataFrame({"review": reviews, "label": labels})
df.head()

,review,label
0,<START> this film was just brilliant casting l...,1
1,<START> big hair big boobs bad music and a gia...,0
2,<START> this has to be one of the worst films ...,0
3,<START> the <UNK> <UNK> at storytelling the tr...,1
4,<START> worst mistake of my life br br i picke...,0


In [16]:
import pandas as pd

# Small dataset of reviews and labels (1 = positive, 0 = negative)
data = {
    "review": [
        "I loved this movie, it was fantastic!",
        "Terrible film. Waste of time.",
        "The product is great and works well.",
        "Worst purchase I've made.",
        "Absolutely amazing! Highly recommended.",
        "Not what I expected. Disappointed."
    ],
    "label": [1, 0, 1, 0, 1, 0]
}

df = pd.DataFrame(data)
df

,review,label
0,"I loved this movie, it was fantastic!",1
1,Terrible film. Waste of time.,0
2,The product is great and works well.,1
3,Worst purchase I've made.,0
4,Absolutely amazing! Highly recommended.,1
5,Not what I expected. Disappointed.,0


In [17]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

# Apply VADER
def vader_sentiment(text):
    score = vader.polarity_scores(text)["compound"]
    return 1 if score >= 0 else 0  # Simple binary classification

df["vader_pred"] = df["review"].apply(vader_sentiment)

In [18]:
from transformers import BertTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Use BERT tokenizer to preprocess text
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Simple tokenizer wrapper
def bert_tokenize(texts):
    return [" ".join(tokenizer.tokenize(t)) for t in texts]

df["bert_tokens"] = bert_tokenize(df["review"])

# TF-IDF vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["bert_tokens"])
y = df["label"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Logistic Regression
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

df.loc[y_test.index, "bert_logreg_pred"] = y_pred


In [13]:
print("VADER Accuracy:", accuracy_score(df["label"], df["vader_pred"]))
print("BERT+LogReg Accuracy:", accuracy_score(y_test, y_pred))

# See predictions side-by-side
df[["review", "label", "vader_pred", "bert_logreg_pred"]]

VADER Accuracy: 1.0
BERT+LogReg Accuracy: 0.5


,review,label,vader_pred,bert_logreg_pred
0,"I loved this movie, it was fantastic!",1,1,1.0
1,Terrible film. Waste of time.,0,0,1.0
2,The product is great and works well.,1,1,NaN
3,Worst purchase I've made.,0,0,NaN
4,Absolutely amazing! Highly recommended.,1,1,NaN
5,Not what I expected. Disappointed.,0,0,NaN
